In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.text import *
#from google.colab import drive
#drive.mount('/content/drive')

bs =64

path= Path("/home/jupyter/data/courtlistener/judges")
textlist = TextList.from_folder(path)
data_lm = (textlist
           #Inputs: all the text files in path
            #.filter_by_folder(include=['train']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs))
data_lm.save('data_lm.pkl')

print('lm done')
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
learn.lr_find()
learn.recorder.plot(skip_end=15)
learn.fit_one_cycle(3, 1e-2, moms=(0.8, 0.7))
learn.save('fit_head')
learn.load('fit_head');
#to complete fine-tuning, unfreeze and launch a new training
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7)) #todo default 10
learn.save('fine_tuned')
learn.load('fine_tuned')
TEXT="In the case at issue, the legal ruling is"
N_WORDS = 40
N_SENTENCES=2
learn.export(file=Path('/home/jupyter/data/sctous_opinion_generator_LSTM_export.pkl'))
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
learn.save_encoder('fine_tuned_enc')

lm done


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


epoch,train_loss,valid_loss,accuracy,time


In [ ]:

TEXT="In my humble opinion"
N_WORDS = 40
N_SENTENCES=2
learn = load_learner(Path('/home/jupyter/data'),'sctous_opinion_generator_LSTM_export.pkl')

print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In my humble opinion , the Court says , No . 8 is " the subject of 246 cases . " Ante , p. 232 . The Court 's opinion requires reconsideration of the Court 's conclusion that
In my humble opinion the Court first finds that the Federal Employers ' Liability Act ( FELA ) , 47 U.S. C. § 185 , provides a remedy for injury to employees and that " the


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.text import *
from google.colab import drive
#drive.mount('/content/drive')
data_lm = load_data(path,'data_lm.pkl', bs=bs) 
#Classifier
path= Path('/home/jupyter/data/courtlistener/judges')
bs=64
classes = os.listdir(path)
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             #.split_by_folder(valid='test')
             .split_by_rand_pct(0.1)
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=classes)
             #label them all with their folders
             .databunch(bs=bs))

data_clas.save('data_clas.pkl')
data_clas.save('data_clas.pkl')
data_clas = load_data(path, 'data_clas.pkl', bs=bs)
#data_clas.show_batch()

#creatae a model to classify those reviews and load the encoder we saved before
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')
learn.lr_find()
learn.recorder.plot()
learn.fit_one_cycle(1, 2e-2, moms=(.8, .7))

learn.save('first')
learn.load('first');
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))
learn.save('second')
learn.load('second')
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))
learn.save('third')
learn.load('third')

learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))
learn.export(file=Path("/content/drive/My Drive/data/sctous_opinion_classifier_export.pkl"))
learn.predict("The holding in the case at hand")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:457: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:537: UserWarning: You are labelling your items with CategoryList.
Your train set contained the following unknown labels, the corresponding items have been discarded.
roberts, matthews
  if getattr(ds, 'warn', False): warn(ds.warn)


epoch,train_loss,valid_loss,accuracy,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


epoch,train_loss,valid_loss,accuracy,time
